In [11]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.options.display.max_rows = 50
pd.options.display.max_columns = None
import matplotlib as plt
import seaborn as sns
import json
from ibm_watson import AssistantV1
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [22]:
input_txt = pd.read_excel('./input.xlsx')
input_txt.to_csv('csvfile.csv', encoding='utf-8', index=False)
input_txt[:10]

,Input,Output,Intent,Confidence
0,hi,NaN,NaN,NaN
1,hello,NaN,NaN,NaN
2,hey,NaN,NaN,NaN
3,안녕,NaN,NaN,NaN
4,안녕하세요,NaN,NaN,NaN
5,너희 가게 소개해줘,NaN,NaN,NaN
6,피자 교환하고 싶어요,NaN,NaN,NaN
7,메뉴 소개,NaN,NaN,NaN
8,주문 취소하고 싶어요,NaN,NaN,NaN
9,가격 알려주세요,NaN,NaN,NaN


# API - V2

In [16]:
# API key
# 3czvIudA1LbpbeWJnLXx4fF4XszwO64JMUYULRvTCR1W
authenticator = IAMAuthenticator('UNqYEG2iIJnZAn_xVLPQkOdNI3W9YuMV3tmHaPQf1M6B')
assistant = AssistantV2(
    # fixed
    version='2020-02-05',
    authenticator = authenticator
)

# resource url : "assistant-21" page...No assistant settings page
# https://api.kr-seo.assistant.watson.cloud.ibm.com/instances/0aaba7a6-94a6-4cd5-beb6-3b518468fa36
assistant.set_service_url('https://api.kr-seo.assistant.watson.cloud.ibm.com/instances/c372bad0-90eb-4df8-bf8b-d873620a20ac')

# "assistant settings" page
session_id = assistant.create_session(
    # 02431287-25f8-4a54-ae15-dac563b2006f
    assistant_id='a3bc3abb-bc4d-4e12-9770-de6ecdeaf1a0'
).get_result()

session_id['session_id']

'd07e46bf-f17c-49ca-bbfc-aa308d5be21c'

In [17]:
def get_response(input_):
    # 02431287-25f8-4a54-ae15-dac563b2006f
    response = assistant.message(
        assistant_id='a3bc3abb-bc4d-4e12-9770-de6ecdeaf1a0',
        session_id=session_id['session_id'],
        input={
            'message_type': 'text',
            'text': input_
        }
    ).get_result()
    
    if len(response['output']['intents']) == 0:
        return response['output']['generic'][0]['text'], 'No Intent', 'No Intent'
    
    return response['output']['generic'][0]['text'], response['output']['intents'][0]['intent'], int(response['output']['intents'][0]['confidence'] * 100)

In [18]:
import time
start_time = time.time()

input_txt['Output'], input_txt['Intent'], input_txt['Confidence'] = zip(*input_txt['Input'].apply(lambda x : get_response(x)))

# 동일 input에 대해서도 시간 차이 꽤 있다. ex) 5개 input에 대해서 1.5~3.7s
print(f"{input_txt.shape[0]}개 input, 수행시간 : {(int((time.time() - start_time)*10))/10}초")

100개 input, 수행시간 : 29.1초


In [19]:
input_txt.to_excel('./output_txt.xlsx', index=False)
input_txt

,Input,Output,Intent,Confidence
0,hi,안녕하세요!,인사,100
1,hello,네 반가워요!,인사,100
2,hey,알아듣기 쉽게 다시 말씀해주시면 좋겠어요.,No Intent,No Intent
3,안녕,안녕하세요!,인사,100
4,안녕하세요,네 반가워요!,인사,100
5,너희 가게 소개해줘,칭찬해주신거죠? 감사해요.,칭찬,38
6,피자 교환하고 싶어요,감사합니다. 피자 주문 도와드릴까요?,주문,79
7,메뉴 소개,다시 말씀해주시겠어요? 이해가 잘 안 되네요.,No Intent,No Intent
8,주문 취소하고 싶어요,"네, 주문 취소합니다.",취소,87
9,가격 알려주세요,전 피자배달봇 피달입니다.<br>잘 부탁드립니다!,누구,30


# Note


1. Intent 안잡히는 경우 존재 함. 그럴 경우 접근할 때 error 발생 => 없는 경우에 대해서만 예외 처리 추가


2. 100개 예문에 대해 output 뽑을 시, 약 27초 소요. 알고리즘 개선할 수 있는 부분 없고, 예문이 많을 수록 선형 증가할 뿐


3. Session 시간 제한 있는 것 같음(분 단위)


4. 승현님이 Node AK_Plaza input과 관련된 node 만들어주기로 함


5. 파이썬으로 시각화 객체 던질 수 있는 라이브러리 확인